## Load Modules

In [1]:
import torch
import pandas as pd
from efficientnet_pytorch import EfficientNet
from datetime import date
import glob
from PIL import Image
from torchvision import  transforms,datasets, models
import torch.nn as nn

c:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Private variables

In [2]:
PATH_WEIGHTS = "D:/ML_exercise/Weights"
folderNames = ['0','1','2','3','4','5','6','7','8','9','a','b','c','d','e','f']
OUTPUT_CSV_PATH = "D:/ML_exercise/PublicTesting_OutputCsv"
#local_dir = "/work/u2785625/AI_Cup_2022/Datasets"
local_dir = "D:/ML_exercise/Test_dataset"
n_classes = 33
PAR = 0.1
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
species = ['asparagus', 'bambooshoots', 'betel', 'broccoli', 'cauliflower', 'chinesecabbage', 'chinesechives', 'custardapple', 'grape', 'greenhouse', 'greenonion', 'kale', 'lemon', 'lettuce', 'litchi', 'longan', 'loofah', 'mango', 'onion', 'others', 'papaya', 'passionfruit', 'pear', 'pennisetum', 'redbeans', 'roseapple', 'sesbania', 'soybeans', 'sunhemp', 'sweetpotato', 'taro', 'tea', 'waterbamboo']

## Testing Transformation

In [3]:
test_tfm = transforms.Compose([ #valid and grading
    transforms.ToTensor(),
    transforms.Resize((456,456)),
    transforms.CenterCrop(456),
    transforms.Normalize([.5, 0.5, 0.5], [0.5, 0.5, 0.5]), 
])

#data = datasets.ImageFolder(root = local_dir, transform = test_tfm)

## Load Model

In [4]:
model = models.efficientnet_b5(weights='DEFAULT')
num_ftrs = model.classifier[1].in_features
model.fc = nn.Linear(num_ftrs, n_classes)
model.load_state_dict(torch.load(PATH_WEIGHTS, map_location=torch.device('cpu')))


<All keys matched successfully>

## Prediction

In [19]:
model.eval()
model.to(device)
#IMG_PATH = "D:/ML_exercise/Test_dataset/0/000bbbfa-15f7-43ba-b99e-0aacada1c782.jpg"
IMG_PATH = "C:/Users/user/Downloads/318758239_649285323564273_108982210974634080_n.jpg"
#Final_label_index = [1, 2, 10, 11, 16, 17, 19, 20, 29, 30]
Final_label_index = [0, 1, 2, 3, 5, 10, 11, 15, 16, 17, 19, 20, 23, 26, 29, 30]
img = Image.open(IMG_PATH)
x = test_tfm(img).to(device)
x = x.unsqueeze(0)
output = model(x)
#print(output)
_, pred = torch.max(output, dim = 1)
percentage = nn.functional.softmax(output, dim=1)[0]
perc = percentage[int(pred)].item()
print(perc)
result = species[pred]
print(result)

# 得到预测结果，并且从大到小排序
_, indices = torch.sort(output, descending=True)
# 返回每个预测值的百分数
print([(species[idx], percentage[idx].item()) for idx in indices[0][:3]])


1.0
custardapple
[('custardapple', 1.0), ('pear', 2.6621430393447554e-08), ('mango', 5.297584149133172e-09)]


In [20]:
for i in range(len(Final_label_index)):
    output[0][Final_label_index[i]] += PAR

#print(output)
_, pred = torch.max(output, dim = 1)
percentage = nn.functional.softmax(output, dim=1)[0]
perc = percentage[int(pred)].item()
print(perc)
result = species[pred]
print(result)

# 得到预测结果，并且从大到小排序
_, indices = torch.sort(output, descending=True)
# 返回每个预测值的百分数
print([(species[idx], percentage[idx].item()) for idx in indices[0][:3]])

1.0
custardapple
[('custardapple', 1.0), ('pear', 2.6621430393447554e-08), ('mango', 5.8547269254916046e-09)]


In [14]:
model.eval()
model.to(device)
List_FileName = list()
List_Output = list()
for folder in folderNames:
    #print(folder)
    folderPath = str(local_dir)+ '/'  + folder + '/'
    for fileName in glob.glob(folderPath + '*'):
        #print(fileName)
        img = Image.open(fileName)
        x = test_tfm(img)
        x = x.unsqueeze(0).to(device)
        
        output = model(x)
        for i in range(len(Final_label_index)):
            output[0][Final_label_index[i]] += PAR
        _, pred = torch.max(output, dim = 1)
        #print(pred)
        #print(fileName[len(folderPath)::])
        #print(species[pred])
        List_FileName.append(fileName[len(folderPath)::])
        List_Output.append(species[pred])

In [17]:
output_dict = {"filename": List_FileName,
                "label": List_Output}
Out_dataframe = pd.DataFrame(output_dict)
print(Out_dataframe)

                                       filename           label
0      0005055c-aaaa-4e19-9a9b-23f8fbbe0976.jpg        soybeans
1      0006061f-abda-4e2b-a07f-506ce19a0efc.jpg      greenonion
2      00069669-5fc6-46fb-91c1-d9d7abba85ea.jpg     waterbamboo
3      000bbbfa-15f7-43ba-b99e-0aacada1c782.jpg  chinesecabbage
4      001787cb-d749-4e9b-be38-eca87ba0efa8.jpg            pear
...                                         ...             ...
11143  ffef051e-044c-4e30-9067-4d840a0252e6.jpg          litchi
11144  ffefbeee-d580-4aba-bdc8-eecd9cee4591.jpg         lettuce
11145  ffefe03e-842c-44cb-a7c2-4649f42742c8.jpg          longan
11146  fff5eae5-791c-4d2b-b501-868f443f90f1.jpg      greenonion
11147  fff9637a-544f-4dc4-b64f-ba92ebfe075e.jpg           grape

[11148 rows x 2 columns]


## Save Csv

In [22]:
FileName =  date.today().strftime("%m/%d").replace('/','') + '.csv'
PAR_VALUE = "_PAR=" + str(PAR) + "_Date:"
print(OUTPUT_CSV_PATH + FileName )
Out_dataframe.to_csv(OUTPUT_CSV_PATH + FileName, index = False)

D:/ML_exercise/PublicTesting_OutputCsv1215.csv
